In [11]:
%load_ext autoreload
%autoreload 2
import sys
import os

while not os.getcwd().endswith('image_drift_generator'):
    os.chdir('..')
    print(os.getcwd())

from image_drift_generator.image_generator import ImageDatasetGenerator
from image_drift_generator.image_factory import *
import warnings
import polars as pl

warnings.filterwarnings('ignore')
# from alibi_detect.cd import MMDDrift
# set tight layout

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
img_path = '../../data/places/'
seed = 42
num_samples = 32
compute_embeddings = True

In [ ]:
image_generator = ImageDatasetGenerator(
    seed=seed, input_path=img_path, compute_embeddings=compute_embeddings
)

In [27]:
all_transform = [
    TransformInfo(
        transf_type=ImageTransform.ROTATE,
        drift_level=0.9,
    ),
    TransformInfo(
        transf_type=ImageTransform.BRIGHTNESS,
        drift_level=0.3,
    ),
    TransformInfo(
        transf_type=ImageTransform.CONTRAST,
        drift_level=0.1,
    ),
    TransformInfo(
        transf_type=ImageTransform.HUE,
        drift_level=0.1,
    ),
    TransformInfo(
        transf_type=ImageTransform.SATURATION,
        drift_level=0.5,
    ),
    TransformInfo(
        transf_type=ImageTransform.GAUSSIAN_BLUR,
        drift_level=0.1,
    ),
    TransformInfo(
        transf_type=ImageTransform.GAUSSIAN_NOISE,
        drift_level=1.0,
    ),
]

In [28]:
image_generator.add_abrupt_drift(
    transform_list=all_transform,
)

In [29]:
image_generator.transform_pipeline

Compose(
      RandomRotation(degrees=[-81.0, 81.0], interpolation=InterpolationMode.NEAREST, expand=False, fill=0)
      ColorJitter(brightness=(0.10000000000000009, 1.9))
      ColorJitter(contrast=(0.7, 1.3))
      ColorJitter(hue=(-0.05, 0.05))
      ColorJitter(saturation=(0.0, 8.5))
      GaussianBlur(kernel_size=(5, 5), sigma=[0.30000000000000004, 0.30000000000000004])
      GaussianNoise(mean=0.1, sigma=0.5, clip=True)
)

In [ ]:
data_category = image_generator.sample(
    num_samples, output_path='data/places/places_generated/'
)

2025-02-17 00:07:21.949 | INFO     | image_drift_generator.image_generator:_make_archive:364 - Archive created successfully: data/places/places_generated/sampled_images.zip
February 17, 2025 > 00:07:21 | INFO | Archive created successfully: data/places/places_generated/sampled_images.zip | {}
2025-02-17 00:07:21.954 | INFO     | image_drift_generator.image_generator:_make_archive:374 - Original folder 'data/places/places_generated/sampled_images' deleted after zipping.
February 17, 2025 > 00:07:21 | INFO | Original folder 'data/places/places_generated/sampled_images' deleted after zipping. | {}


In [31]:
print(data_category.target)

shape: (32, 3)
┌───────────┬───────────┬───────┐
│ sample-id ┆ timestamp ┆ label │
│ ---       ┆ ---       ┆ ---   │
│ i64       ┆ f64       ┆ i64   │
╞═══════════╪═══════════╪═══════╡
│ 0         ┆ 1.7172e9  ┆ 0     │
│ 1         ┆ 1.7172e9  ┆ 1     │
│ 2         ┆ 1.7172e9  ┆ 1     │
│ 3         ┆ 1.7172e9  ┆ 1     │
│ 4         ┆ 1.7172e9  ┆ 1     │
│ …         ┆ …         ┆ …     │
│ 27        ┆ 1.7173e9  ┆ 0     │
│ 28        ┆ 1.7173e9  ┆ 0     │
│ 29        ┆ 1.7173e9  ┆ 1     │
│ 30        ┆ 1.7173e9  ┆ 0     │
│ 31        ┆ 1.7173e9  ┆ 0     │
└───────────┴───────────┴───────┘


In [32]:
print(data_category.input_mapping)

shape: (32, 3)
┌───────────┬───────────┬─────────────────────────────────┐
│ sample-id ┆ timestamp ┆ file_name                       │
│ ---       ┆ ---       ┆ ---                             │
│ i64       ┆ f64       ┆ str                             │
╞═══════════╪═══════════╪═════════════════════════════════╡
│ 0         ┆ 1.7172e9  ┆ e67a7559-5aad-4714-88dd-871b55… │
│ 1         ┆ 1.7172e9  ┆ e2a5dce3-a12f-4cb2-a61a-775dd8… │
│ 2         ┆ 1.7172e9  ┆ 0b2ad164-fa92-4a93-a782-416867… │
│ 3         ┆ 1.7172e9  ┆ a6d55d63-3b42-4eb7-99d6-7771ec… │
│ 4         ┆ 1.7172e9  ┆ 0760a200-8f48-4b86-a734-ef3036… │
│ …         ┆ …         ┆ …                               │
│ 27        ┆ 1.7173e9  ┆ 520e6397-37fa-4521-a596-92b87d… │
│ 28        ┆ 1.7173e9  ┆ 28336beb-7ea7-43d4-9935-48ceaf… │
│ 29        ┆ 1.7173e9  ┆ 4ae3421d-f4be-47ad-aeea-ebc52e… │
│ 30        ┆ 1.7173e9  ┆ f321bc90-12fb-4590-aa5f-07ab5f… │
│ 31        ┆ 1.7173e9  ┆ 38979374-9df7-4607-861a-d794eb… │
└───────────┴───────────┴

In [33]:
print(data_category.input_embedding_file_path)

data/places/places_generated/embeddings.parquet


In [34]:
print(pl.read_parquet(data_category.input_embedding_file_path))

shape: (32, 3)
┌───────────┬───────────┬─────────────────────────────────┐
│ sample-id ┆ timestamp ┆ embedding                       │
│ ---       ┆ ---       ┆ ---                             │
│ i64       ┆ f64       ┆ array[f64, 512]                 │
╞═══════════╪═══════════╪═════════════════════════════════╡
│ 0         ┆ 1.7172e9  ┆ [0.646854, 0.737511, … 0.13151… │
│ 1         ┆ 1.7172e9  ┆ [0.395607, 0.753779, … 0.00640… │
│ 2         ┆ 1.7172e9  ┆ [0.124174, 0.690031, … 0.00171… │
│ 3         ┆ 1.7172e9  ┆ [0.491539, 0.826048, … 0.05988… │
│ 4         ┆ 1.7172e9  ┆ [0.381547, 0.728992, … 0.35658… │
│ …         ┆ …         ┆ …                               │
│ 27        ┆ 1.7173e9  ┆ [0.240201, 0.385757, … 0.00122… │
│ 28        ┆ 1.7173e9  ┆ [0.234493, 0.968768, … 0.0]     │
│ 29        ┆ 1.7173e9  ┆ [1.592411, 1.461076, … 0.05754… │
│ 30        ┆ 1.7173e9  ┆ [0.226647, 0.556803, … 0.12092… │
│ 31        ┆ 1.7173e9  ┆ [0.069852, 0.384815, … 0.05021… │
└───────────┴───────────┴

In [10]:
print(data_category.input_folder_type)
print(data_category.input_folder)
print(data_category.input_file_type)
print(data_category.is_input_folder)
print(data_category.is_target_folder)


zip
c:\Users\crist\projects\image_drift_generator\data\places\places_generated\sampled_images.zip
png
True
False
